# Checking Sentiment Analysis Result by CSAssistant

no of sample = 75

* The sentiment analysis generated by CSAssistant will be compared with the benchmark answer.
* Then compute the accuracy.

In [1]:
import config
from openai import OpenAI
import json

client = OpenAI()

sentiment_analysis_template = """
    # CONTEXT #
    You are a ticket assistant for a bus transport company, CasuewayLink.
    You are responsible for performing sentiment analysis on the customer tickets.
    
    # OBJECTIVE #
    You will be given a ticket and learn the context in the ticket and then perform sentiment analysis. 
    You need to provide the sentiment result of the query, such as:
        - Neutral
        - Positive
        - Negative

    # STYLE #
    Ensure sentiment analysis is accurate and consistent.
    
    # TONE # 
    Neutral
    
    # AUDIENCE #
    CausewayLink's customer support team
    
    # RESPONSE #
    Your output must strictly follow the below format in JSON format:

    {"Sentiment": "Sentiment Analysis Result..."}
    
    For example:
    
    {"Sentiment": "Positive"}
    
    """

def analyze_ticket(query: str, templates: list) -> str:
    """
    Perform topic modelling, summarization and sentiment analysis on the given ticket.
    
    Args:
        query (str): User ticket's description_text
        tempaltes (list): The template in list for analyzing ticket.

    Returns:
        JSON (dict): Topic Modelling, Summarization and Sentiment Analysis results in JSON format.
    """
    output = {} # initiate empty dictionary
    
    for template in templates:
      response=client.chat.completions.create(
          model="gpt-4o",
          messages=[
              {"role":"system","content":template},
              {"role":"user","content":query}
                    ],
          response_format={ "type": "json_object" },
      )
      
      result = json.loads(response.choices[0].message.content) # convert to json format (dict in Python)
      
      output =  output | result # merge dictionaries
      
    return output

In [2]:
query = "Okay thanks, ahve a great day!"
analyze_ticket(query, [sentiment_analysis_template])

{'Sentiment': 'Positive'}

In [3]:
tickets = open('results.json')

tickets = json.load(tickets)


In [4]:
sentiment_result_json = {int(k):{"Sentiment": v['Sentiment']} for k,v in tickets.items()}
sentiment_result_json

{0: {'Sentiment': 'Neutral'},
 1: {'Sentiment': 'Neutral'},
 2: {'Sentiment': 'Negative'},
 3: {'Sentiment': 'Neutral'},
 4: {'Sentiment': 'Negative'},
 5: {'Sentiment': 'Positive'},
 6: {'Sentiment': 'Neutral'},
 13: {'Sentiment': 'Neutral'},
 14: {'Sentiment': 'Neutral'},
 16: {'Sentiment': 'Neutral'},
 17: {'Sentiment': 'Neutral'},
 23: {'Sentiment': 'Neutral'},
 26: {'Sentiment': 'Negative'},
 27: {'Sentiment': 'Negative'},
 28: {'Sentiment': 'Negative'},
 29: {'Sentiment': 'Neutral'},
 30: {'Sentiment': 'Negative'},
 32: {'Sentiment': 'Negative'},
 33: {'Sentiment': 'Negative'},
 69: {'Sentiment': 'Negative'},
 70: {'Sentiment': 'Neutral'},
 75: {'Sentiment': 'Neutral'},
 78: {'Sentiment': 'Neutral'},
 87: {'Sentiment': 'Negative'},
 89: {'Sentiment': 'Negative'},
 100: {'Sentiment': 'Neutral'},
 106: {'Sentiment': 'Neutral'},
 141: {'Sentiment': 'Neutral'},
 144: {'Sentiment': 'Neutral'},
 158: {'Sentiment': 'Negative'},
 161: {'Sentiment': 'Negative'},
 175: {'Sentiment': 'Negat

In [5]:
validation_json = {int(k):analyze_ticket(v['Query'], [sentiment_analysis_template]) for k,v in tickets.items()}
validation_json

{0: {'Sentiment': 'Neutral'},
 1: {'Sentiment': 'Neutral'},
 2: {'Sentiment': 'Negative'},
 3: {'Sentiment': 'Neutral'},
 4: {'Sentiment': 'Negative'},
 5: {'Sentiment': 'Neutral'},
 6: {'Sentiment': 'Neutral'},
 13: {'Sentiment': 'Neutral'},
 14: {'Sentiment': 'Neutral'},
 16: {'Sentiment': 'Neutral'},
 17: {'Sentiment': 'Neutral'},
 23: {'Sentiment': 'Neutral'},
 26: {'Sentiment': 'Negative'},
 27: {'Sentiment': 'Negative'},
 28: {'Sentiment': 'Neutral'},
 29: {'Sentiment': 'Neutral'},
 30: {'Sentiment': 'Negative'},
 32: {'Sentiment': 'Negative'},
 33: {'Sentiment': 'Negative'},
 69: {'Sentiment': 'Negative'},
 70: {'Sentiment': 'Neutral'},
 75: {'Sentiment': 'Neutral'},
 78: {'Sentiment': 'Neutral'},
 87: {'Sentiment': 'Negative'},
 89: {'Sentiment': 'Negative'},
 100: {'Sentiment': 'Neutral'},
 106: {'Sentiment': 'Neutral'},
 141: {'Sentiment': 'Neutral'},
 144: {'Sentiment': 'Neutral'},
 158: {'Sentiment': 'Negative'},
 161: {'Sentiment': 'Negative'},
 175: {'Sentiment': 'Negativ

In [7]:
n = len(sentiment_result_json)
diff_count = 0
different_keys = []

for key in sentiment_result_json.keys():
    if key in validation_json.keys():
        if sentiment_result_json[key] != validation_json[key]:
            diff_count += 1
            different_keys.append(key)

print(f'The sentiment analysis by CSAssistant is {(n-diff_count)/n * 100:.2f}%')
print(diff_count)
print(different_keys)

The sentiment analysis by CSAssistant is 86.67%
10
[5, 28, 230, 246, 257, 648, 732, 902, 966, 1373]
